# Notebook Info

Transferring files from the backup Db to a database in production

Database Locations
```
Backup 
DataBase: oasis-data
Schema: oasis_dbo  -- Newer
Schema: xspoc_dbo  -- Older

Production
Database: oasis-prod
Schema: xspoc
```

Following tables are to be transferred:
```
tblCardData
tblXDiagRodResults
tblXDiagResults
tblWellTests
```

While transferring data, use both the older and newer backups to get the entire historical data. Drop Duplicates if necessary

# Imports

In [1]:
"""
For setting up local imports in an Ipython Shell
This is a workaround for ipython, dont need it for basic python scripts
"""
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
# local
from library import lib_aws, lib_dyna

# Basic
import pandas as pd
import numpy as np
import time

# Geometry
from shapely.wkb import loads
from geoalchemy2.shape import to_shape

# options
pd.set_option('display.max_rows', 1000)

# tblCardData

This data will be stored in production (`oasis-prod`) in schema `xspoc` in table `card`

In [46]:
# Basic data anaysis
# querying dates in new backup
query = """
select 
distinct("NodeID"),
min("Date") as min_date,
max("Date") as max_date
from oasis_dbo."tblCardData"
group by "NodeID"
order by "NodeID"
"""

with lib_aws.PostgresRDS(db='oasis-data') as engine:
    curr_info = pd.read_sql(query, engine)

Connected to oasis-data DataBase
Connection Closed


In [53]:
cutoff_end = pd.Timestamp('2020-08-28 07:00:00')  # Data after this is erroroneous
# for now we wont use start dates

{'Dale 5601 14-26B'}

In [5]:
%%time
# Distinct wells in card data
query_new = """select distinct("NodeID") from oasis_dbo."tblCardData" order by "NodeID";"""
query_old = """select distinct("NodeID") from xspoc_dbo."tblCardData" order by "NodeID";"""

with lib_aws.PostgresRDS(db="oasis-data") as engine:
    well_new = pd.read_sql(query_new, engine)
    well_old = pd.read_sql(query_old, engine)

Connected to oasis-data DataBase
Connection Closed
Wall time: 1min 36s


- As the dataset is huge, with a lot of card columns. One well at a time will be transferred.
- In addition, looking at the initial analysis the newer backup has all the data we need, se we dont need to worry about the older backup data

Following steps need to be done:
- Query from newer backup
- Query only columns needed
- Clean up well name
- Convert card columns from hex to wkb (`lib_dyna.hex_to_wkb()`)
- Remove duplicate and wrong timestamps


After analysing the wells the new backup has the following additional wells
```
'Berry 5493 41-7 6B',
'Berry 5493 42-7 9B',
'Berry 5493 44-7 15TX',
'Crane Federal 5300 14-27 3B',
'Dawson 5494 42-12 3B',
'Lewis Federal 5300 11-31 2B',
'Lewis Federal 5300 11-31 4BR',
'Lite Federal 5393 41-11 13BX',
'Mcfarland Federal 5501 11-18 8BX',
'Om Erickson 5501 11-18 4B',
'Osprey 44-23H',
'Stenberg 5199 11-9 9B'
```

Additionally following are the wells which have errorneous data-points
```
'Behan 2-29H',
'Hanson 33-28H',
'Holm 14X-12H',
'K2 Holdings',
'Knaebel 41-11T',
'Loren 1-12H',
'MONTAGUE 5601 41-34 6T',
'MONTAGUE 5601 41-34 7B',
'Nevis 12-18H'
```

In [60]:
def transfer_backup_card(well_name):
    start = time.time()
    # Max time supposed to be in data
    backup_max_time = pd.Timestamp('2020-08-28 07:00:00')

    # query for well
    query = """
    select 
    "NodeID",
    "Date",
    "AnalysisDate",
    "SPM",
    "StrokeLength",
    "Runtime",
    "FillBasePct",
    "Fillage",
    "SecondaryPumpFillage",
    encode("tblCardData"."POCDownholeCardB", 'hex') as "POCDownholeCardB",
    encode("tblCardData"."SurfaceCardB", 'hex') as "SurfaceCardB",
    encode("tblCardData"."DownholeCardB", 'hex') as "DownholeCardB",
    encode("tblCardData"."PredictedCardB", 'hex') as "PredictedCardB",
    encode("tblCardData"."TorquePlotMinEnergyB", 'hex') as "TorquePlotMinEnergyB",
    encode("tblCardData"."TorquePlotMinTorqueB", 'hex') as "TorquePlotMinTorqueB",
    encode("tblCardData"."PermissibleLoadUpB", 'hex') as "PermissibleLoadUpB"
    from oasis_dbo."tblCardData"
    where "NodeID" = '{}'
    order by "Date";
    """.format(well_name)

    # Import from backup
    with lib_aws.PostgresRDS(db="oasis-data") as engine:
        test_data = pd.read_sql(query, engine)

    # Basic Cleaning
    test_data.fillna(np.nan, inplace=True)
    test_data['Date'] = pd.to_datetime(test_data['Date'])
    test_data['AnalysisDate'] = pd.to_datetime(test_data['AnalysisDate'])
    # Cleaning Well Names
    test_data["NodeID"] = (test_data["NodeID"].str.replace("#", "")  # remove #
                                            .str.replace('\s+', ' ', regex=True)  # remove multiple spaces if present
                                            .str.strip()  # Remove trailing whitespaces
                                            .str.lower()  # lower all character
                                            .str.title()  # Uppercase first letter of each word
                                            .map(lambda x: x[0:-2] + x[-2:].upper()))  # last 2 characters should alwsy be upper case

    # Dropping Wrong Dates
    test_data = test_data[test_data.Date <= backup_max_time]
    test_data.drop_duplicates(subset=['NodeID', 'Date'], inplace=True)
    test_data.reset_index(inplace=True, drop=True)

    # Converting Card cols from hex to wkb
    card_cols = [
        'POCDownholeCardB',
        'SurfaceCardB',
        'DownholeCardB',
        'PredictedCardB',
        'TorquePlotMinEnergyB',
        'TorquePlotMinTorqueB',
        'PermissibleLoadUpB'
    ]
    
    try:
        for col in card_cols:  # Converting the hex columns to a wkb format
            test_data.loc[:, col] = test_data.loc[:, col].apply(lib_dyna.CardFunctions.hex_to_wkb)
    except Exception as e:
        print(e)
        print(well_name)
        return print("Issue with card col")
    
    end = time.time()
    print('Time taken for import and cleaning: {:.2f}s'.format(end-start))
    print('NodeID name changed from {} to {}'.format(well_name, test_data.loc[0, "NodeID"]))
    print("Total data points {}".format(test_data.shape[0]))
    print(f'Max time: {test_data.Date.max()}')
    
    # Add data to production
#     lib_aws.AddData.add_data(df=test_data,
#                             db='oasis-prod',
#                             schema='xspoc',
#                             table='card',
#                             merge_type='append',
#                             card_col=card_cols,
#                             index_col='Date')
    return test_data

In [65]:
df = transfer_backup_card('Angell 31-28H')

# Use the python script for running it for the entire dataset

Connected to oasis-data DataBase
Connection Closed
Time taken for import and cleaning: 80.48s
NodeID name changed from Angell 31-28H to Angell 31-28H
Total data points 6648
Max time: 2020-08-17 06:21:34


# XDiagRodResults

In [ ]:
# Distinct wells in card data
query = """
select distinct("NodeID") from oasis_dbo."tblXDiagRodResults" order by "NodeID";
"""

with lib_aws.PostgresRDS(db="oasis-data") as engine:
    wells = pd.read_sql(query, engine)

In [ ]:
%%time
full_query = """select * from oasis_dbo."tblXDiagRodResults" order by "NodeID", "Date";"""

with lib_aws.PostgresRDS(db="oasis-data") as engine:
    xdiagrod = pd.read_sql(full_query, engine)

In [ ]:
# Basic Cleaning
xdiagrod["NodeID"] = (xdiagrod["NodeID"].str.replace("#", "")  # remove #
                                       .str.replace('\s+', ' ', regex=True)  # remove multiple spaces if present
                                       .str.strip()  # Remove trailing whitespaces
                                       .str.lower()  # lower all character
                                       .str.title()  # Uppercase first letter of each word
                                       .map(lambda x: x[0:-2] + x[-2:].upper()))  # last 2 characters should always be upper case


In [ ]:
xdiagrod.groupby(["NodeID"]).agg({"Date": [min, max, "count"]})

In [ ]:
xdiagrod.drop_duplicates(subset=['NodeID', 'Date', 'RodNum'], inplace=True)
xdiagrod.sort_values(by=['NodeID', 'Date'], inplace=True)
xdiagrod.reset_index(inplace=True, drop=True)

In [ ]:
# Add Data to DB
lib_aws.AddData.add_data(df=xdiagrod,
                        db='oasis-prod',
                        schema='xspoc',
                        table='xdiagrod',
                        merge_type='append',
                        card_col=None,
                        index_col='Date')

# XDiagResults

In [ ]:
# Distinct wells in card data
query = """
select distinct("NodeID") from oasis_dbo."tblXDiagResults" order by "NodeID";
"""

with lib_aws.PostgresRDS(db="oasis-data") as engine:
    wells = pd.read_sql(query, engine)

In [ ]:
%%time
full_query = """select * from oasis_dbo."tblXDiagResults" order by "NodeID", "Date";"""

with lib_aws.PostgresRDS(db="oasis-data") as engine:
    xdiag = pd.read_sql(full_query, engine)

In [ ]:
# Basic Cleaning
xdiag["NodeID"] = (xdiag["NodeID"].str.replace("#", "")  # remove #
                                       .str.replace('\s+', ' ', regex=True)  # remove multiple spaces if present
                                       .str.strip()  # Remove trailing whitespaces
                                       .str.lower()  # lower all character
                                       .str.title()  # Uppercase first letter of each word
                                       .map(lambda x: x[0:-2] + x[-2:].upper()))  # last 2 characters should always be upper case


In [ ]:
old_info = xdiag.groupby(["NodeID"]).agg({"Date": [min, max, "count"]})

In [ ]:
xdiag.drop_duplicates(subset=['NodeID', 'Date'], inplace=True)
xdiag.sort_values(by=['NodeID', 'Date'], inplace=True)
xdiag.reset_index(inplace=True, drop=True)

In [ ]:
# Add Data to DB
lib_aws.AddData.add_data(df=xdiag,
                        db='oasis-prod',
                        schema='xspoc',
                        table='xdiag',
                        merge_type='append',
                        card_col=None,
                        index_col='Date')

## Importing older data

This data is imported from the older backup that was sent.
```
Details:
Database = oasis-data
schema = xspoc_dbo
table = tblXDiagResults
```

- Will have to be done in a well specific basis.
- Query min and max values from production database and drop the additional dates accordingly

In [81]:
# querying dates in new backup
query = """
select 
distinct("NodeID"),
min("Date") as min_date,
max("Date") as max_date
from xspoc."xdiag"
group by "NodeID"
order by "NodeID"
"""

with lib_aws.PostgresRDS(db='oasis-prod') as engine:
    curr_info = pd.read_sql(query, engine)

Connected to oasis-prod DataBase
Connection Closed


In [82]:
curr_info

,NodeID,min_date,max_date
0,Aagvik 1-35H,2019-06-21 15:58:34,2020-07-21 10:18:00
1,Acadia 31-25H,2019-05-27 23:33:12,2020-08-15 16:18:15
2,Acklins 12-18H,2019-05-27 23:52:43,2020-08-15 15:29:25
3,Aerabelle 5502 43-7T,2019-05-27 23:49:54,2020-08-15 02:39:51
4,Ak Strangeland 43-12T,2019-05-28 01:20:59,2020-08-15 12:27:00
5,Al 44-23B,2019-05-27 23:12:48,2020-08-15 16:03:22
6,Alder 43-8H,2019-05-27 23:35:26,2020-08-15 14:02:37
7,Amazing Grace Federal 11-2H,2019-06-12 00:26:11,2020-08-14 09:24:29
8,Amelia Federal 41-11B,2019-05-28 00:27:03,2020-08-15 10:13:49
9,Anders Madson 14-25 11B,2019-05-28 13:38:55,2020-08-15 15:08:08


In [75]:
def transfer_xdiag_data(well_name, cut_off):
    specific_query = """
    SELECT * FROM xspoc_dbo."tblXDiagResults"
    WHERE "NodeID" = '{}' 
    AND "Date" < '{}'
    """.format(well_name, cut_off)

    with lib_aws.PostgresRDS(db="oasis-data") as engine:
        test_data = pd.read_sql(specific_query, engine)

    # Basic Cleaning
    test_data["NodeID"] = (test_data["NodeID"].str.replace("#", "")  # remove #
                                               .str.replace('\s+', ' ', regex=True)  # remove multiple spaces if present
                                               .str.strip()  # Remove trailing whitespaces
                                               .str.lower()  # lower all character
                                               .str.title()  # Uppercase first letter of each word
                                               .map(lambda x: x[0:-2] + x[-2:].upper()))  # last 2 characters should always be upper case

    test_data.drop_duplicates(subset=['NodeID', 'Date'], inplace=True)
    test_data.sort_values(by=['NodeID', 'Date'], inplace=True)
    test_data.reset_index(inplace=True, drop=True)

    try:
        # Add Data to DB
        lib_aws.AddData.add_data(df=test_data,
                                db='oasis-prod',
                                schema='xspoc',
                                table='xdiag',
                                merge_type='append',
                                card_col=None,
                                index_col='Date')
    except Exception as e:
        print(e)
        return print("Data Already Present (Unique Keys match)")
    
    try:
        print("WellName is {}".format(test_data.iloc[0,0]))
    except Exception as e:
        print("No Data")
        
    return None

In [79]:
for i in curr_info.index[74:]:
    well = curr_info.loc[i, "NodeID"]
    cut = curr_info.loc[i, "min_date"].strftime("%Y-%m-%d %H:%M:%S")
    print('\n--------------')
    print(i)
    print(f'WellName: {well} and Cut-off Date: {cut}')
    transfer_xdiag_data(well, cut)


--------------
74
WellName: BERRY 5493 42-7 9B and Cut-off Date: 2020-06-17 07:35:43
Connected to oasis-data DataBase
Connection Closed
Connected to oasis-prod DataBase
Connection Closed
Data appended on Table xdiag in time 4.14s
No Data

--------------
75
WellName: BERRY 5493 44-7 12T and Cut-off Date: 2020-02-16 13:18:13
Connected to oasis-data DataBase
Connection Closed
Connected to oasis-prod DataBase
Connection Closed
Data appended on Table xdiag in time 5.58s
WellName is Berry 5493 44-7 12T

--------------
76
WellName: BERRY 5493 44-7 15TX and Cut-off Date: 2020-06-29 11:42:18
Connected to oasis-data DataBase
Connection Closed
Connected to oasis-prod DataBase
Connection Closed
Data appended on Table xdiag in time 4.56s
No Data

--------------
77
WellName: BERWICK 4-2HE and Cut-off Date: 2020-02-16 00:43:38
Connected to oasis-data DataBase
Connection Closed
Connected to oasis-prod DataBase
Connection Closed
Data appended on Table xdiag in time 10.00s
WellName is Berwick 4-2HE

--

Connection Closed
Data appended on Table xdiag in time 9.69s
WellName is Brier 44-22 6T2

--------------
106
WellName: Brier 44-22 7B and Cut-off Date: 2020-02-15 23:38:58
Connected to oasis-data DataBase
Connection Closed
Connected to oasis-prod DataBase
Connection Closed
Data appended on Table xdiag in time 10.04s
WellName is Brier 44-22 7B

--------------
107
WellName: Brier 5200 43-22 12T and Cut-off Date: 2020-02-16 02:30:16
Connected to oasis-data DataBase
Connection Closed
Connected to oasis-prod DataBase
Connection Closed
Data appended on Table xdiag in time 9.14s
WellName is Brier 5200 43-22 12T

--------------
108
WellName: BRODERSON 13-35H and Cut-off Date: 2020-02-16 01:20:23
Connected to oasis-data DataBase
Connection Closed
Connected to oasis-prod DataBase
Connection Closed
Data appended on Table xdiag in time 9.26s
WellName is Broderson 13-35H

--------------
109
WellName: BRODERSON 2-27H and Cut-off Date: 2020-02-16 01:46:58
Connected to oasis-data DataBase
Connection C

Connection Closed
Connected to oasis-prod DataBase
Connection Closed
Data appended on Table xdiag in time 9.86s
WellName is Ceynar 11-28B

--------------
137
WellName: Ceynar 29-32H and Cut-off Date: 2020-02-17 08:25:03
Connected to oasis-data DataBase
Connection Closed
Connected to oasis-prod DataBase
Connection Closed
Data appended on Table xdiag in time 7.62s
WellName is Ceynar 29-32H

--------------
138
WellName: Ceynar 4-18HA and Cut-off Date: 2020-02-16 01:12:57
Connected to oasis-data DataBase
Connection Closed
Connected to oasis-prod DataBase
Connection Closed
Data appended on Table xdiag in time 7.36s
WellName is Ceynar 4-18HA

--------------
139
WellName: Ceynar 4-18HB and Cut-off Date: 2020-02-16 00:26:28
Connected to oasis-data DataBase
Connection Closed
Connected to oasis-prod DataBase
Connection Closed
Data appended on Table xdiag in time 7.21s
WellName is Ceynar 4-18HB

--------------
140
WellName: Ceynar 4X-18H and Cut-off Date: 2020-02-15 23:57:12
Connected to oasis-da

Connection Closed
Connected to oasis-prod DataBase
Connection Closed
Data appended on Table xdiag in time 7.28s
WellName is Cook Federal 41-12 10BX

--------------
169
WellName: Cookie Federal 24-25B and Cut-off Date: 2020-02-15 23:15:34
Connected to oasis-data DataBase
Connection Closed
Connected to oasis-prod DataBase
Connection Closed
Data appended on Table xdiag in time 7.98s
WellName is Cookie Federal 24-25B

--------------
170
WellName: Cooper 5603 43-8H and Cut-off Date: 2020-02-16 08:37:08
Connected to oasis-data DataBase
Connection Closed
Connected to oasis-prod DataBase
Connection Closed
Data appended on Table xdiag in time 7.50s
WellName is Cooper 5603 43-8H

--------------
171
WellName: COPPER 2959 42-20H and Cut-off Date: 2020-02-16 01:26:49
Connected to oasis-data DataBase
Connection Closed
Connected to oasis-prod DataBase
Connection Closed
Data appended on Table xdiag in time 7.28s
WellName is Copper 2959 42-20H

--------------
172
WellName: CORNELL 5501 13-1H and Cut-of

Connection Closed
Data appended on Table xdiag in time 12.57s
WellName is Delia 14-30H

--------------
200
WellName: Delta 6093 14-15 10T and Cut-off Date: 2020-02-16 03:59:33
Connected to oasis-data DataBase
Connection Closed
Connected to oasis-prod DataBase
Connection Closed
Data appended on Table xdiag in time 10.22s
WellName is Delta 6093 14-15 10T

--------------
201
WellName: Delta 6093 14-15 11B and Cut-off Date: 2020-02-15 23:29:36
Connected to oasis-data DataBase
Connection Closed
Connected to oasis-prod DataBase
Connection Closed
Data appended on Table xdiag in time 7.76s
WellName is Delta 6093 14-15 11B

--------------
202
WellName: Delta 6093 14-15 9B and Cut-off Date: 2020-02-16 02:02:55
Connected to oasis-data DataBase
Connection Closed
Connected to oasis-prod DataBase
Connection Closed
Data appended on Table xdiag in time 7.25s
WellName is Delta 6093 14-15 9B

--------------
203
WellName: Delta 6093 24-15 2T and Cut-off Date: 2020-02-16 10:02:32
Connected to oasis-data D

...

Connection Closed
Data appended on Table xdiag in time 20.99s
WellName is Shaw 11-23 3B

--------------
735
WellName: Shaw 11-23 5T2 and Cut-off Date: 2020-02-16 01:57:45
Connected to oasis-data DataBase
Connection Closed
Connected to oasis-prod DataBase
Connection Closed
Data appended on Table xdiag in time 11.57s
WellName is Shaw 11-23 5T2

--------------
736
WellName: Shaw 11-23 6B and Cut-off Date: 2020-02-16 00:08:31
Connected to oasis-data DataBase
Connection Closed
Connected to oasis-prod DataBase
Connection Closed
Data appended on Table xdiag in time 10.96s
WellName is Shaw 11-23 6B

--------------
737
WellName: Shaw 11-23 7T and Cut-off Date: 2020-02-16 02:04:00
Connected to oasis-data DataBase
Connection Closed
Connected to oasis-prod DataBase
Connection Closed
Data appended on Table xdiag in time 9.26s
WellName is Shaw 11-23 7T

--------------
738
WellName: Shaw 24-22H and Cut-off Date: 2020-02-16 00:57:00
Connected to oasis-data DataBase
Connection Closed
Connected to oasis

Connection Closed
Connected to oasis-prod DataBase
Connection Closed
Data appended on Table xdiag in time 6.88s
WellName is Spratley Federal 5494 44-13 3BX

--------------
766
WellName: Stacey 13-9H and Cut-off Date: 2020-02-15 23:47:33
Connected to oasis-data DataBase
Connection Closed
Connected to oasis-prod DataBase
Connection Closed
Data appended on Table xdiag in time 7.33s
WellName is Stacey 13-9H

--------------
767
WellName: State 5792 11-15 2T and Cut-off Date: 2020-02-16 00:59:22
Connected to oasis-data DataBase
Connection Closed
Connected to oasis-prod DataBase
Connection Closed
Data appended on Table xdiag in time 10.35s
WellName is State 5792 11-15 2T

--------------
768
WellName: State 5792 11-15H and Cut-off Date: 2020-02-15 23:59:11
Connected to oasis-data DataBase
Connection Closed
Connected to oasis-prod DataBase
Connection Closed
Data appended on Table xdiag in time 11.94s
WellName is State 5792 11-15H

--------------
769
WellName: State 5792 31-15 3B and Cut-off Dat

Connection Closed
Connected to oasis-prod DataBase
Connection Closed
Data appended on Table xdiag in time 8.16s
WellName is Sylvia 44-23T

--------------
798
WellName: Syverson 16-34H and Cut-off Date: 2020-02-16 00:32:48
Connected to oasis-data DataBase
Connection Closed
Connected to oasis-prod DataBase
Connection Closed
Data appended on Table xdiag in time 8.09s
WellName is Syverson 16-34H

--------------
799
WellName: TALEN 2759 43-7H and Cut-off Date: 2020-02-15 23:26:05
Connected to oasis-data DataBase
Connection Closed
Connected to oasis-prod DataBase
Connection Closed
Data appended on Table xdiag in time 11.14s
WellName is Talen 2759 43-7H

--------------
800
WellName: TANA 2759 43-7H and Cut-off Date: 2020-02-16 00:48:53
Connected to oasis-data DataBase
Connection Closed
Connected to oasis-prod DataBase
Connection Closed
Data appended on Table xdiag in time 9.50s
WellName is Tana 2759 43-7H

--------------
801
WellName: Tanner T 5601 42-24B and Cut-off Date: 2020-02-16 00:57:21

Connection Closed
Connected to oasis-prod DataBase
Connection Closed
Data appended on Table xdiag in time 7.37s
WellName is Twobins 5501 41-20 6T

--------------
830
WellName: Twobins 5501 41-20 8T and Cut-off Date: 2020-02-16 00:48:06
Connected to oasis-data DataBase
Connection Closed
Connected to oasis-prod DataBase
Connection Closed
Data appended on Table xdiag in time 7.88s
WellName is Twobins 5501 41-20 8T

--------------
831
WellName: Twobins 5501 41-21 9TX and Cut-off Date: 2020-02-16 01:02:01
Connected to oasis-data DataBase
Connection Closed
Connected to oasis-prod DataBase
Connection Closed
Data appended on Table xdiag in time 8.53s
WellName is Twobins 5501 41-21 9TX

--------------
832
WellName: Twobins 5501 42-20H and Cut-off Date: 2020-02-15 23:16:34
Connected to oasis-data DataBase
Connection Closed
Connected to oasis-prod DataBase
Connection Closed
Data appended on Table xdiag in time 7.27s
WellName is Twobins 5501 42-20H

--------------
833
WellName: Tyler T 5601 41-24B

Connection Closed
Connected to oasis-prod DataBase
Connection Closed
Data appended on Table xdiag in time 7.88s
WellName is Wold Federal 15-33H

--------------
861
WellName: Woll 6093 12-1T and Cut-off Date: 2020-02-19 06:45:35
Connected to oasis-data DataBase
Connection Closed
Connected to oasis-prod DataBase
Connection Closed
Data appended on Table xdiag in time 10.03s
WellName is Woll 6093 12-1T

--------------
862
WellName: Wrangell Federal 5602 43-11H and Cut-off Date: 2020-02-16 00:06:09
Connected to oasis-data DataBase
Connection Closed
Connected to oasis-prod DataBase
Connection Closed
Data appended on Table xdiag in time 8.82s
WellName is Wrangell Federal 5602 43-11H

--------------
863
WellName: Wren Federal 41-26H and Cut-off Date: 2020-02-15 23:10:19
Connected to oasis-data DataBase
Connection Closed
Connected to oasis-prod DataBase
Connection Closed
Data appended on Table xdiag in time 9.79s
WellName is Wren Federal 41-26H

--------------
864
WellName: YEISER 5603 42-33H a